In [4]:

from Experiment import Experiment
import matplotlib.pyplot as plt
import Plotters 
import plotly.graph_objects as go
import plotly
import numpy as np

import time
import matplotlib.cm as colormap

%matplotlib qt

loadir = 'C:/' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis
exp_name =['manipulated_05_12_22']
experiments = {exp:Experiment(loadir,exp) for exp in exp_name} # load experiments


In [11]:
exp = experiments['manipulated_05_12_22']
mov_name = exp.mov_names[0]
mov = exp.get_mov(mov_name)
print(mov.header['body'].keys())
pitch_body = mov.get_prop('pitch_body','body')

dict_keys(['pitch_body', 'yaw_body', 'roll_body', 'pitch_body_dot', 'yaw_body_dot', 'roll_body_dot', 'pitch_body_dot_dot', 'yaw_body_dot_dot', 'roll_body_dot_dot', 'frames', 'time', 'CM_real_x_body', 'CM_real_y_body', 'CM_real_z_body', 'phi_rw_max_idx', 'phi_rw_max_val', 'phi_lw_max_idx', 'phi_lw_max_val', 'phi_rw_min_idx', 'phi_rw_min_val', 'phi_lw_min_idx', 'phi_lw_min_val', 'yaw_z_frame', 'pitch_y_frame', 'roll_x_frame', 'yaw_z_frame_dot', 'pitch_y_frame_dot', 'roll_x_frame_dot', 'yaw_z_frame_dot_dot', 'pitch_y_frame_dot_dot', 'roll_x_frame_dot_dot'])


Smooth, calculate projected axes and angles

In [23]:
[exp.smooth_prop_movies('CM_real_x_body',['','_dot','_dot_dot'],'body')  for exp in experiments.values()] # smooth and derive


[exp.project_axes_movies(['X_x_body_projected','X_y_body_projected'])  for exp in experiments.values()] # X_body axis projected to xy plane
[exp.project_axes_movies(['CM_dot_xax','CM_dot_yax'],axis = 'CM_real_x_body_dot_smth',wing_body = 'body',add_to_vectors = True)  for exp in experiments.values()] # Velocity direction in the projected X_body axes (xy plane) 

# calculate the size of the velocity projected on the xy plane
[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected',add_to_vectors = True)  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_y',three_col = 2,ax_to_proj = 'X_y_body_projected',add_to_vectors = True)  for exp in experiments.values()]


# calculate the size of the acceleration projected on the xy plane
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected')  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y',three_col = 2,ax_to_proj = 'X_y_body_projected')  for exp in experiments.values()]

[exp.pqr_movies() for exp in experiments.values()] # calculate p,q,r (angular velocity in body axis)

[exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi') for exp in experiments.values()] # calculate the mean value of two properties


[exp.substruct_first_frame('yaw_body','body')  for exp in experiments.values()] # substract the zero frame from a property ( {prop}_min_ref_frame )
[exp.velocity_amplitude('CM_dot_x_projected','CM_dot_y_projected','body')  for exp in experiments.values()] # calculate the amplitude of the velocity on xy plane

# calculate the difference between current direction and another one at the same time:
# the angle between Xbody on the xy plane and the velocity direction on the xy plane 
[exp.delta_ang_all_time_movies('X_x_body_projected','CM_dot_xax','slip_angle_beta',three_col = 2) for exp in experiments.values()] 

# calculate the difference in angle between current property and in referance frame:
# the angle between Xbody on the xy (in 0 time) plane and Xbody on xy plane 
[exp.delta_ref_ang_movies('X_x_body_projected','delta_ang_Xax',axis = 'X_x_body_projected')  for exp in experiments.values()]

# the angle between Xbody on the xy (in 0 time) plane and the velocity direction on the xy plane 
[exp.delta_ref_ang_movies('CM_dot_x_projected','delta_ang_CM_dot_x',axis = 'CM_dot_xax',delta_ang = 0) for exp in experiments.values()]


c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:107: RuntimeWarning:

invalid value encountered in arccos



[None, None, None, None, None, None]

In [24]:
exp = experiments['manipulated_2022_02_03_dark']

prop = 'pitch_body'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

Calculate first peak/ min/ max

In [25]:
exp = experiments[list(experiments.keys())[0]]
[exp.get_peaks_movies('pitch_y_frame_dot_dot','peaks_max',t0 = 5, t1 = 50) for exp in experiments.values()]
[exp.get_peaks_movies('pitch_y_frame_dot_dot','min') for exp in experiments.values()]
[exp.get_peaks_movies('q','max') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_body','peaks_max',t0 = 40, t1 = 100) for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_y_frame','max') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_body','max') for exp in experiments.values()]

[exp.get_peaks_movies('CM_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_x_projected','zero_v') for exp in experiments.values()]

[exp.get_peaks_movies('amp_v','peaks_min') for exp in experiments.values()]

[exp.get_peaks_movies('amp_v','peaks_min') for exp in experiments.values()]
[exp.get_peaks_movies('delta_ang_Xax','max') for exp in experiments.values()]


[exp.get_peaks_movies('CM_dot_x_projected','half_v') for exp in experiments.values()]


point_name = 'CM_dot_x_projected_zero_v'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()



point_name = 'CM_dot_dot_x_projected_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()



point_name = 'q_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


In [26]:

def rotation_matrix(yaw,pitch,roll):
    roll_mat = np.vstack([[1,0,0],[0 ,np.cos(roll),-np.sin(roll)],[0, np.sin(roll), np.cos(roll)]])
    pitch_mat = np.vstack([[np.cos(pitch),0,np.sin(pitch)],[0, 1,0],[-np.sin(pitch), 0, np.cos(pitch)]])
    yaw_mat = np.vstack([[np.cos(yaw),-np.sin(yaw),0],[np.sin(yaw),np.cos(yaw),0],[0, 0, 1]])
    return yaw_mat @ pitch_mat @ roll_mat


for exp_name in experiments.keys():
    exp = experiments[exp_name]
    wing_body = 'body'
    for mov_name in exp.mov_names:
        mov = exp.get_mov(mov_name)

        yaw = mov.get_prop('yaw_body','body')[:,0]
        pitch = mov.get_prop('pitch_body','body')[:,0]
        roll = mov.get_prop('roll_body','body')[:,0]
        time = mov.get_prop('time','body')[:,0]

        rot_mat = [rotation_matrix(yaw*np.pi/180,-pitch*np.pi/180,roll*np.pi/180).T for yaw,pitch,roll in zip(yaw,pitch,roll)]
        vxyz = mov.get_prop('CM_real_x_body_dot_smth','body',three_col=3)
        axyz = mov.get_prop('CM_real_x_body_dot_dot_smth','body',three_col=3)

        uvw = np.vstack([rot_mat @ vxyz for rot_mat,vxyz in zip(rot_mat,vxyz)])
        uvw_dot = np.vstack([rot_mat @ axyz for rot_mat,axyz in zip(rot_mat,axyz)])

        vz_sp = uvw[:,0]*np.cos(np.radians(45)) + uvw[:,2]*np.cos(np.radians(45))
        vx_sp = uvw[:,0]*np.cos(np.radians(45)) - uvw[:,2]*np.cos(np.radians(45))



        az_sp = uvw_dot[:,0]*np.cos(np.radians(45)) + uvw_dot[:,2]*np.cos(np.radians(45))
        ax_sp = uvw_dot[:,0]*np.cos(np.radians(45)) - uvw_dot[:,2]*np.cos(np.radians(45))

        mov.data[wing_body] = np.hstack((mov.data[wing_body], uvw))
        mov.add_to_header(['u','v','w'],wing_body)

        
        mov.data[wing_body] = np.hstack((mov.data[wing_body], uvw_dot))
        mov.add_to_header(['u_dot','v_dot','w_dot'],wing_body)

        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, vz_sp)).T
        mov.add_to_header(['vz_sp'],wing_body)
        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, vx_sp)).T
        mov.add_to_header(['vx_sp'],wing_body)

        
        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, az_sp)).T
        mov.add_to_header(['az_sp'],wing_body)
        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, ax_sp)).T
        mov.add_to_header(['ax_sp'],wing_body)
# fig,ax = plt.subplots(2,1)
# ax[0].plot(time,vx_sp)
# ax[0].plot(time,uvw[:,1])
# ax[0].plot(time,vz_sp)

# ax[1].plot(vxyz)
    
# for mov_name in exp.mov_names:

    
    # mov.data[wing_body] = np.hstack((mov.data[wing_body], uvw))
    # mov.add_to_header(['u','v','w'],wing_body)

    # mov.data[wing_body] = np.vstack((mov.data[wing_body].T, asp_z)).T
    # mov.add_to_header(['asp_z'],wing_body)


In [44]:
exp = experiments['manipulated_2022_02_03_dark']

prop = 'CM_dot_x_projected'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

In [28]:
exp = experiments['manipulated_2022_02_03_dark']
prop = 'q'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

In [34]:
exp = experiments['manipulated_2023_08_10_100ms']
prop = 'vx_sp'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

prop = 'vz_sp'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

prop = 'v'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()


prop = 'ax_sp'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

prop = 'az_sp'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

prop = 'v_dot'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()


In [36]:

exp = experiments['manipulated_2022_02_03_dark']
prop = 'u_dot'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

prop = 'w_dot'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()


prop = 'v_dot'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()



prop = 'q'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()


prop = 'asp_x'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()


prop = 'asp_z'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

prop = 'CM_dot_dot_x_projected'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

KeyError: 'asp_x'

In [ ]:

prop = 'az_sp'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time_m_t0')
fig.show()

In [ ]:
for name in ['ax_sp','az_sp','v_dot']:
    f,tm = exp.mean_time_series_prop(name,wing_body,window = (73*7)//2,dt = 1/16000)
    plt.plot(tm,f)

In [ ]:

exp = experiments['manipulated_2023_08_09_60ms']
prop = 'CM_dot_x_projected'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('CM_dot_x_projected_half_v',prop,fig) 
fig.show()



In [42]:
exp = experiments['manipulated_2023_08_09_60ms']
prop_name = 'CM_dot_x_projected'
dict_yaw = {'no_change':[],'change':[]}
for mov_nm in exp.mov_names:
    mov = exp.get_mov(mov_nm)
    prop = mov.get_prop(prop_name,'body')
    if np.nanmean(prop[0:200])  > 0:
        dict_yaw['change'].append(mov_nm )
    else:
        dict_yaw['no_change'].append(mov_nm )

fig = go.Figure()

exp.plot_prop_movies(prop_name,'body',fig,mov = dict_yaw['change'],case = 'plot_exp',prop_x = 'time',color = color_map[3],legend = f'cluster{1}')
exp.plot_prop_movies(prop_name,'body',fig,mov = dict_yaw['no_change'],case = 'plot_exp',prop_x = 'time',color = color_map[4],legend = f'cluster{2}')
fig.show()

prop_name = 'CM_dot_y_projected'
fig = go.Figure()

exp.plot_prop_movies(prop_name,'body',fig,mov = dict_yaw['change'],case = 'plot_exp',prop_x = 'time',color = color_map[3],legend = f'cluster{1}')
exp.plot_prop_movies(prop_name,'body',fig,mov = dict_yaw['no_change'],case = 'plot_exp',prop_x = 'time',color = color_map[4],legend = f'cluster{2}')
fig.show()



prop_name = 'pitch_body_dot_dot'
fig = go.Figure()

exp.plot_prop_movies(prop_name,'mean_body',fig,mov = dict_yaw['change'],case = 'plot_exp',prop_x = 'time',color = color_map[3],legend = f'cluster{1}')
exp.plot_prop_movies(prop_name,'mean_body',fig,mov = dict_yaw['no_change'],case = 'plot_exp',prop_x = 'time',color = color_map[4],legend = f'cluster{2}')
fig.show()


In [ ]:
time_idx = exp.body_header['time']
prop_to_move = 'pitch_body_peaks_max'
plt.figure()
plt.hist(exp.interest_points['CM_dot_x_projected_zero_v'][:,time_idx] - exp.interest_points['pitch_y_frame_dot_dot_peaks_max'][:,time_idx], alpha =0.5, label = 'delta_zerov_pitch_dotdot')
plt.hist(exp.interest_points['pitch_y_frame_dot_dot_peaks_max'][:,time_idx], alpha = 0.5,label = 'pitch_dotdot')
plt.hist(exp.interest_points['CM_dot_x_projected_zero_v'][:,time_idx], alpha = 0.5,label = 'zero_v')
plt.legend()


plt.figure()
plt.hist(exp.interest_points['CM_dot_x_projected_zero_v'][:,time_idx] - exp.interest_points['CM_dot_x_projected_min'][:,time_idx], alpha =0.5, label = 'delta_zerov_cm_dot')
plt.hist(exp.interest_points['CM_dot_x_projected_min'][:,time_idx], alpha = 0.5,label = 'min_cm_dot')
plt.hist(exp.interest_points['CM_dot_x_projected_zero_v'][:,time_idx], alpha = 0.5,label = 'zero_v')
plt.legend()


plt.figure()
plt.hist(exp.interest_points['pitch_y_frame_dot_dot_peaks_max'][:,time_idx] - exp.interest_points['CM_dot_x_projected_zero_v'][:,time_idx] , alpha =0.5, label = 'delta_zerov_pitch_dotdot')
plt.hist(exp.interest_points['CM_dot_x_projected_min'][:,time_idx] - exp.interest_points['CM_dot_x_projected_zero_v'][:,time_idx] , alpha =0.5, label = 'delta_zerov_min_cm_dot')
plt.legend()




plt.figure()
plt.hist(exp.interest_points['CM_dot_x_projected_zero_v'][:,time_idx] - exp.interest_points['pitch_body_max'][:,time_idx], alpha =0.5, label = 'delta_zerov_pitch_body_max')
plt.hist(exp.interest_points['pitch_body_max'][:,time_idx], alpha = 0.5,label = 'pitch_body_max')
plt.hist(exp.interest_points['CM_dot_x_projected_zero_v'][:,time_idx], alpha = 0.5,label = 'zero_v')
plt.legend()




In [ ]:
plt.figure()
time_idx = exp.body_header['time']

for exp_nm in  exp_name[:-1]:
    exp = experiments[exp_nm]

    v0_list = []
    time_of_zero_v = exp.interest_points['CM_dot_x_projected_zero_v'][:,time_idx]

    for mov_name in exp.mov_names:
        mov = exp.get_mov(mov_name)
        v0_list.append(mov.get_prop('CM_dot_x_projected','body')[mov.ref_frame,:])

    plt.plot(v0_list,time_of_zero_v,'*',label = exp_nm)
plt.legend()


In [ ]:

exp_name =['manipulated_2023_08_10_100ms',
           'manipulated_2022_02_03_dark',
           'manipulated_2023_08_06_40ms',
           'manipulated_2023_08_09_60ms',
           'manipulated_2023_08_09_80ms',
           'manipulated_2023_08_07_10ms']
prop = 'slip_angle_beta'#'slip_angle_beta'#'slip_angle_beta'
time_idx = exp.body_header['time']
fig,ax = plt.subplots(2,1)
wing_body = 'body'
for expnm in exp_name: 
    exp = experiments[expnm]

    for idx,mov in enumerate(exp.mov_names):
            
            t0 = exp.interest_points['CM_dot_x_projected_zero_v'][idx][exp.body_header['time']]
            mov = exp.get_mov(mov)
            prop_vals = mov.get_prop(prop,'body')
            time = mov.get_prop('time','body')
            mov.data[wing_body] = np.hstack((mov.data[wing_body], time - t0))
            mov.add_to_header([f'time_m_t0'],wing_body)
            # ax[0].plot(time - t0,prop_vals)
            # ax[0].title.set_text('moved to zero v')
            # ax[1].plot(time,prop_vals)
            # ax[1].title.set_text('original time')

            


            # mov.data[wing_body] = np.hstack((mov.data[wing_body], time - t0))
            # mov.add_to_header([f'time_m_t0'],wing_body)
            
        # t0 = exp.interest_points['CM_dot_x_projected_zero_v'][idx][exp.body_header['time']]


In [ ]:
fig = go.Figure()
exp = experiments['manipulated_2023_08_09_60ms']
prop = 'v'
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time_m_t0') 
fig.show()


In [ ]:
fig = go.Figure()
prop = 'CM_dot_dot_x_projected'
[exp.plot_prop_movies(prop,'body',fig,case = 'plot_exp',add_horizontal_line = 0,prop_x = 'time',color = color_map[idx],legend = exp.pertubation_name) for idx,exp in enumerate(experiments.values())]
fig.show()

In [ ]:
fig = go.Figure()
prop = 'slip_angle_beta'
[exp.plot_prop_movies(prop,'body',fig,case = 'plot_exp',prop_x = 'time_m_t0',color = color_map[idx],legend = exp.pertubation_name) for idx,exp in enumerate(experiments.values())]
fig.show()


KeyError: 'time_m_t0'

Change in axis (delta angle) compared to its projection in t = 0

In [ ]:

exp_name =['manipulated_2023_08_10_100ms',
           'manipulated_2022_02_03_dark',
           'manipulated_2023_08_06_40ms',
           'manipulated_2023_08_09_60ms',
           'manipulated_2023_08_09_80ms',
           'manipulated_2023_08_07_10ms']

for expn in exp_name:
    exp = experiments[expn]
    fig = go.Figure()
    exp.plot_prop_movies('delta_ang_CM_dot_x','vectors',fig,case = 'plot_mov',prop_x = 'time',add_horizontal_line = None)
    fig.update_layout(title = expn)
    fig.show()


In [ ]:
exp = experiments['manipulated_2023_08_09_60ms']
fig = go.Figure()
exp.plot_prop_movies('slip_angle_beta','body',fig,case = 'plot_mov',prop_x = 'time_m_t0',add_horizontal_line = None)
fig.show()


In [ ]:
fig = go.Figure()
prop = 'slip_angle_beta'
[exp.plot_prop_movies(prop,'body',fig,case = 'plot_exp' ,prop_x = 'time_m_t0',color = color_map[idx],legend = exp.pertubation_name) for idx,exp in enumerate(experiments.values())]
fig.show()

In [ ]:
exp = experiments['manipulated_2023_08_09_60ms']
fig = go.Figure()
exp.plot_prop_movies('delta_ang_Xax','body',fig,case = 'plot_mov',add_horizontal_line = None )

fig.show()

In [ ]:
exp = experiments['manipulated_2023_08_09_60ms']
fig = go.Figure()
exp.plot_prop_movies('slip_angle_beta','vectors',fig,case = 'plot_mov',prop_x = 'time',add_horizontal_line = None)
fig.show()
exp = experiments['manipulated_2023_08_09_60ms']
fig = go.Figure()
exp.plot_prop_movies('delta_ang_Xax','vectors',fig,case = 'plot_mov',prop_x = 'time',add_horizontal_line = None)
fig.show()



In [ ]:
exp = experiments['manipulated_2023_08_10_100ms']
fig = go.Figure()
exp.plot_prop_movies('slip_angle_beta','vectors',fig,case = 'plot_mov',prop_x = 'time',add_horizontal_line = None)
fig.show()

fig = go.Figure()
exp.plot_prop_movies('slip_angle_beta','vectors',fig,case = 'plot_mov',prop_x = 'time')
fig.show()
fig = go.Figure()
exp.plot_prop_movies('yaw_z_frame','body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()
fig = go.Figure()
exp.plot_prop_movies('delta_ang_Xax','vectors',fig,case = 'plot_mov',prop_x = 'time',add_horizontal_line = None)
fig.show()




In [ ]:
fig = go.Figure()
exp = experiments['manipulated_2022_02_03_dark']
exp.plot_prop_movies('CM_dot_dot_y_projected','body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)

fig = go.Figure()
exp = experiments['manipulated_2022_02_03_dark']
exp.plot_prop_movies('phi_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
exp.plot_prop_movies('phi_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)

fig.show()

3D trajectory plot

In [ ]:

exp = experiments['manipulated_2023_08_09_60ms']
mov = ['mov60']
exp.plot_3d_traj_movies('CM_dot_x_projected',mov = mov)


In [ ]:
mov = exp.get_mov('mov75')
mov.from_wing_body_to_vectors('slip_angle_beta','vectors', wing_body_to_vectors = False)

In [ ]:
t0 = exp.interest_points['CM_dot_x_projected_zero_v'][idx][exp.body_header['time']]
time[time > t0]

array([ 77.625 ,  77.6875,  77.75  , ..., 331.5   , 331.5625, 331.625 ])

In [ ]:
plt.hist(exp.interest_points['slip_angle_beta_peaks_max'][:,time_idx], alpha = 0.5,label = 'min_cm_dot')


(array([20.,  8.,  4.,  1.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([50.0625 , 50.26875, 50.475  , 50.68125, 50.8875 , 51.09375,
        51.3    , 51.50625, 51.7125 , 51.91875, 52.125  ]),
 <BarContainer object of 10 artists>)

In [ ]:
print(exp.experiment_name)
mov = exp.get_mov('mov94')
prop = mov.get_prop('CM_dot_x_projected','body')
plt.plot(prop)

manipulated_2023_08_09_60ms


In [ ]:
for exp_name in experiments:
    exp = experiments[exp_name]
    time = exp.interest_points['pitch_y_frame_max'][:,exp.body_header['time']]
    max_pitch = exp.interest_points['pitch_y_frame_max'][:,exp.body_header['pitch_body']]

    plt.plot(time,
            max_pitch,'*',label = exp_name)
plt.legend()

Plot histograms of peaks, plot movies time seires data with the max/min/peak data point on them

In [ ]:
# Histograms
point_name = 'pitch_y_frame_dot_dot_peaks_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()

point_name = 'pitch_y_frame_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_dot_projected_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'amp_v_peaks_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_projected_zero_v'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()

In [ ]:
exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('CM_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('amp_v_peaks_min','amp_v',fig) 
fig.show()

In [ ]:
# time series of movies with min/max on them
exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_peaks_max','pitch_y_frame_dot_dot',fig) 
fig.show()

fig = go.Figure()
exp.plot_prop_movies('CM_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_peaks_max','CM_dot_projected',fig) 
fig.show()

In [ ]:



# time series of movies with min/max on them
exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_peaks_max','pitch_y_frame_dot_dot',fig) 
fig.show()

fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_min','pitch_y_frame',fig) 
fig.show()


fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_max','pitch_y_frame_dot_dot',fig) 
fig.show()


fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('CM_dot_dot_projected_min','CM_dot_dot_projected',fig) 
fig.show()


fig = go.Figure()
exp.plot_prop_movies('yaw_z_frame','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('CM_dot_projected_zero_v','yaw_z_frame',fig) 
fig.show()


# 


In [ ]:

fig = go.Figure()
exp.plot_prop_movies('roll_x_frame','body',fig,case = 'plot_mov',prop_x = 'time')
# exp.add_point_to_plot_movies('CM_dot_projected_zero_v','CM_dot_y_projected',fig) 
exp.plot_prop_movies('yaw_z_frame','body',fig,case = 'plot_mov',prop_x = 'time',yaxis = 'y2')
# exp.add_point_to_plot_movies('CM_dot_projected_zero_v','roll_body',fig,yaxis = 'y2') 

# yax_dict[f"yaxis2"] = (dict(title=prop,titlefont=dict(color=color),tickfont=dict(color=color),side = side[i],position=position[i]))



fig.update_layout(title = exp.experiment_name,
    xaxis=dict(domain=[0, 0.95]),
    yaxis=dict(title='roll_x_frame', titlefont=dict(color='red'), tickfont=dict(color='red'), side='left', position=0),
    yaxis2=dict(title='yaw_y_frame', titlefont=dict(color='green'), tickfont=dict(color='green'), side='right', position=0.95),
)

fig.show()

Plot all experiments - time series

In [ ]:
fig = go.Figure()
prop = 'pitch_y_frame_dot_dot'
[exp.plot_prop_movies(prop,'body',fig,case = 'plot_exp',prop_x = 'time',color = color_map[idx],legend = exp.pertubation_name) for idx,exp in enumerate(experiments.values())]
fig.show()

Calculate and plot mean property (time series,  mean per frame of all movies)

In [ ]:
from scipy import stats


exp = experiments[list(experiments.keys())[0]]
prop = 'pitch_y_frame'
prop = 'r'
prop = 'CM_dot_dot_projected'
props = ['delta_ang','delta_ang']

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True)
max_prop = {}
for idx,prop in enumerate(props):
    max_prop[prop] = [] 
    for exp in experiments.values():
        mov_to_keep = []
        acc_mean,time_inter = exp.mean_time_series_prop(prop,'vectors')
        time_to_check = (time_inter > 0 ) & (time_inter <100 )
        time_for_max = time_inter[time_to_check]
        max_prop[prop].append(time_for_max[np.nanargmax(acc_mean[time_to_check])])
        ax[idx].plot(time_inter,acc_mean, label = exp.experiment_name)


plt.legend()

g:\Documents\experiment_analysis\data_analysis\Experiment.py:121: RuntimeWarning:

Mean of empty slice



Try PCA

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.cluster import KMeans

mov_feat = []
props = ['CM_dot_dot_projected','CM_dot_projected','pitch_y_frame_dot_dot']

exp_pd = []
for exp in list(experiments.values())[0:3]:
    mov_feat = []
    for idx,mov_name in enumerate(exp.mov_names):
        mov = exp.get_mov(mov_name)
        mov_feat.append(np.nanmean(mov.data['body'][15:60,:],axis = 0))

    featpd_exp = pd.DataFrame(data = mov_feat,columns = mov.header['body'])
    featpd_exp['exp_name'] = exp.experiment_name
    featpd_exp['mov'] = exp.mov_names
    exp_pd.append(featpd_exp)

feat = pd.concat(exp_pd,axis = 0).dropna().reset_index()


dataforpca = feat[props]
n_comp = 2
pca = PCA(n_components=n_comp)
principalComponents = pca.fit_transform(dataforpca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = [f'pca_{idx}' for idx in range(n_comp)])

pca_name = [f'pca_{idx}' for idx in range(pca.n_components_)]


featpd = pd.concat([feat, principalDf], axis = 1)
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(featpd[pca_name])
print(kmeans.labels_)
pca.components_


C:\Users\Roni\AppData\Local\Temp\ipykernel_31184\2630784800.py:13: RuntimeWarning:

Mean of empty slice

C:\Users\Roni\AppData\Local\Temp\ipykernel_31184\2630784800.py:13: RuntimeWarning:

Mean of empty slice

C:\Users\Roni\AppData\Local\Temp\ipykernel_31184\2630784800.py:13: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



[0 0 0 0 0 0 0 2 2 2 0 0 2 1 0 0 1 0 2 2 2 2 2 0 0 2 2 2 0 0 0 2 0 0 2 2 0
 0 2]


array([[ 2.58930919e-03, -3.21593360e-07, -9.99996648e-01],
       [-9.99996645e-01, -7.69076209e-05, -2.58930916e-03]])

In [ ]:

fig = go.Figure()

for lbl in np.unique(kmeans.labels_):
    exp_names = np.unique(feat.iloc[kmeans.labels_==lbl]['exp_name'])
    feat_lbl = feat.iloc[kmeans.labels_==lbl]
    for idx,exp_name in enumerate(exp_names): 
        mov_exp_lbl = list(feat_lbl.loc[feat_lbl['exp_name'] == exp_name]['mov'])

        prop = 'pitch_y_frame_dot'
        prop = 'CM_dot_dot_projected'
        # prop = 'pitch_body'

        color_map = colormap.datad["tab10"]['listed']

        experiments[exp_name].plot_prop_movies(prop,'body',fig,mov = mov_exp_lbl,case = 'plot_exp',prop_x = 'time',color = color_map[lbl],legend = f'{exp_name}_cluster{lbl}')
fig.show()
# feat.iloc[kmeans.labels_==lbl]['exp_name'] == exp_names[0]
from scipy import stats


exp = experiments[list(experiments.keys())[0]]
prop = 'pitch_y_frame'
prop = 'q'
prop = 'CM_dot_dot_projected'
props = ['pitch_y_frame_dot_dot','pitch_y_frame_dot']

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True)
max_prop = {}
for idx,prop in enumerate(props):
    max_prop[prop] = [] 
    for exp in experiments.values():
        mov_to_keep = []
        acc_mean,time_inter = exp.mean_time_series_prop(prop,'body')
        time_to_check = (time_inter > 0 ) & (time_inter <100 )
        time_for_max = time_inter[time_to_check]
        max_prop[prop].append(time_for_max[np.nanargmax(acc_mean[time_to_check])])
        ax[idx].plot(time_inter,acc_mean)

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Experiment.py:112: RuntimeWarning:

Mean of empty slice



Plot and save figures

In [ ]:
exp

In [ ]:
import plotly.graph_objects as go
import plotly
import matplotlib.cm as colormap
color_map = colormap.datad["tab10"]['listed']



def plot_and_save_movie_prop(exp,yprop,wing_body,title,add_horizontal_line = None):
    
    fig = go.Figure()

    exp.plot_prop_movies(yprop,wing_body,fig,case = 'plot_mov',prop_x = 'time',add_horizontal_line = add_horizontal_line)
    fig.update_layout(title = title)
   
    fig.show()
    plotly.offline.plot(fig, filename=f'{exp.figures_path}/{yprop}_plot.html',auto_open=False)




for exp in experiments:
    try:
        exp = experiments[exp]
        exp.plot_3d_traj_movies('CM_dot_dot_x_projected',save_plot = True, mov = False)

        plot_and_save_movie_prop(exp, 'slip_angle_beta', 'vectors','dot product between Xbody_xy and V_xy', add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'delta_ang_CM_dot_x','vectors','dot product between initial V_xy and V_xy',add_horizontal_line = None)
        plot_and_save_movie_prop(exp, 'delta_ang_Xax','vectors','dot product between initial Xbody_xy and Xbody_xy',add_horizontal_line = None)

        plot_and_save_movie_prop(exp, 'CM_dot_dot_x_projected','acceleration projected on XY',wing_body = 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'pitch_y_frame','pitch angle rotated to first frame', 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'pitch_y_frame_dot','pitch_dot rotated to first frame' 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'CM_dot_x_projected','velocity projected on XY', 'body',add_horizontal_line = 0)
    except:
        continue
    # exp.mean_mean_props_movies('phi_rw_min_val','phi_lw_min_val','mean_wing','mean_front_phi')
    # exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi')

    # plot_and_save_movie_prop(exp,yprop = 'mean_front_phi',wing_body = 'mean_wing')
    # plot_and_save_movie_prop(exp,yprop = 'mean_mean_phi',wing_body = 'mean_wing')

